In [1]:
import random
import numpy as np
import math
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
import re

In [2]:
# class Particle():
#     def __init__(self):
#         self.position = np.array([(-1) ** (bool(random.getrandbits(1))) 
#                                   * random.random() * 50, 
#                                   (-1)**(bool(random.getrandbits(1)))
#                                  * random.random()*50])
#         self.pbest_position = self.position
#         self.pbest_value = float('inf')
#         self.velocity = np.array([0, 0])
    
#     def __str__(self):
#         print("I am at ", self.position, " meu pbest is ", self.pbest_position)
    
#     def move(self):
#         self.position = self.position + self.velocity
    
# class Space():
    
#     def __init__(self, target, target_error, n_particles):
#         self.target = target
#         self.target_error = target_error
#         self.n_particles = n_particles
#         self.particles = []
        
#         self.gbest_value = float('inf')
#         self.gbest_position = np.array([random.random() * 50, random.random() * 50])
    
#     def print_particles(self):
#         for particle in self.particles:
#             particle.__str__()
    
#     def fitness(self, particle):
#         return particle.position[0] ** 2 + particle.position[1] ** 2 + 1
    
#     def set_pbest(self):
#         for particle in self.particles:
#             fitness_cadidate = self.fitness(particle)
            
#             if(particle.pbest_value > fitness_cadidate):
#                 particle.pbest_value = fitness_cadidate
#                 particle.pbest_position = particle.position
    
#     def set_gbest(self):
#         for particle in self.particles:
#             best_fitness_cadidate = self.fitness(particle)
#             if(self.gbest_value > best_finess_cadidate):
#                 self.gbest_value = best_finess_cadidate
#                 self.gbest_position = particle.position
                
#     def move_particles(self):
#         for particle in self.particles:
#             global W
#             new_velocity = (W*particle.velocity) + (c1*random.random()) * (particle.pbest_position - particle.position) + \
#                             (random.random()*c2) * (self.gbest_position - particle.position)
#             particle.velocity = new_velocity
#             particle.move()
        

In [3]:
def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))

In [4]:
# 
def sigmoid_function(particle):
    return 1 / (1+math.exp(-particle))

particle = [1, 0, 0, 0, 1, 0, 1, 0]
sigmoid = sigmoid_function(-1)
sigmoid

0.2689414213699951

In [5]:
def fitness_function(particle_position, X, y, x_test, y_test):
    particle_position = particle_position.tolist()
    
    if(particle_position.count(0) != len(particle_position)):
        # get index with value 0
        cols = [index for index in range(
            len(particle_position)) if particle_position[index] == 0]

        # get features subset
        X_parsed = X.drop(X.columns[cols], axis=1)
        x_test_parsed = x_test.drop(X.columns[cols], axis=1)
        X_subset = pd.get_dummies(X_parsed)
        x_test_subset = pd.get_dummies(x_test_parsed)
        
        # apply classification algorithm
        clf = DecisionTreeClassifier().fit(X_subset, y)
        y_pred = clf.predict(x_test_subset)
        score = f1_score(y_pred, y_test, average='macro')
        print(score)
        mse = mean_squared_error(y_test, y_pred)
        
        # print(particle_position , " -> ", mse)
        # return (avg(cross_val_score(clf, X_subset, y, cv=5)),)
        # return particle_position[0]**2 + particle_position[1]**2 + 1
        return np.sqrt(mse)
    else:
        return(1)
    
    #return particle_position[0]**2 + particle_position[1]**2 + 1

In [6]:
def iris():
    df = pd.read_csv('datasets/iris.csv',)
    le = LabelEncoder()
    le.fit(df.iloc[:, -1])
    y = le.transform(df.iloc[:, -1]) # label
    X = df.drop([df.columns[0], 'Species'], axis=1)
    return X, y

In [7]:
def wine():
    df_red = pd.read_csv('datasets/winequality_red.csv')
    df_white = pd.read_csv('datasets/winequality_white.csv')
    df_red['color'] = "R"
    df_white['color'] = "W"
    print(df_red.size)
    print(df_white.size)
    df = pd.concat([df_red, df_white])

    le = LabelEncoder()
    le.fit(df.iloc[:, -1])
    y = le.transform(df.iloc[:, -1]) # label
    X = df.drop([df.columns[0], 'color'], axis=1)
    return X, y

In [8]:
def titanic():
    train = pd.read_csv('datasets/titanic_train.csv')
    test = pd.read_csv('datasets/titanic_test.csv')
    
    full_data = [train, test]
    
    PassengerId = test['PassengerId']
    # Some features of my own that I have added in
    # Gives the length of the name
    train['Name_length'] = train['Name'].apply(len)
    test['Name_length'] = test['Name'].apply(len)
    # Feature that tells whether a passenger had a cabin on the Titanic
    train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
    test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

    # Feature engineering steps taken from Sina
    # Create new feature FamilySize as a combination of SibSp and Parch
    for dataset in full_data:
        dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    # Create new feature IsAlone from FamilySize
    for dataset in full_data:
        dataset['IsAlone'] = 0
        dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    # Remove all NULLS in the Embarked column
    for dataset in full_data:
        dataset['Embarked'] = dataset['Embarked'].fillna('S')
    # Remove all NULLS in the Fare column and create a new feature CategoricalFare
    for dataset in full_data:
        dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
    train['CategoricalFare'] = pd.qcut(train['Fare'], 4)
    # Create a New feature CategoricalAge
    for dataset in full_data:
        age_avg = dataset['Age'].mean()
        age_std = dataset['Age'].std()
        age_null_count = dataset['Age'].isnull().sum()
        age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
        dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
        dataset['Age'] = dataset['Age'].astype(int)
    train['CategoricalAge'] = pd.cut(train['Age'], 5)
    # Define function to extract titles from passenger names
    def get_title(name):
        title_search = re.search(' ([A-Za-z]+)\.', name)
        # If the title exists, extract and return it.
        if title_search:
            return title_search.group(1)
        return ""
    # Create a new feature Title, containing the titles of passenger names
    for dataset in full_data:
        dataset['Title'] = dataset['Name'].apply(get_title)
    # Group all non-common titles into one single grouping "Rare"
    for dataset in full_data:
        dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

        dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
        dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
        dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

    for dataset in full_data:
        # Mapping Sex
        dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)

        # Mapping titles
        title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
        dataset['Title'] = dataset['Title'].map(title_mapping)
        dataset['Title'] = dataset['Title'].fillna(0)

        # Mapping Embarked
        dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

        # Mapping Fare
        dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
        dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
        dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
        dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
        dataset['Fare'] = dataset['Fare'].astype(int)

        # Mapping Age
        dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
        dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
        dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
        dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
        dataset.loc[ dataset['Age'] > 64, 'Age'] = 4 ;
    # Feature selection
    drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
    train = train.drop(drop_elements, axis = 1)
    train = train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)
    test  = test.drop(drop_elements, axis = 1)
    le = LabelEncoder()
    le.fit(train.iloc[:, 1])
    y = le.transform(train.iloc[:, 1])
    X = train.drop([train.columns[0], 'Survived'], axis=1)
  
    return X, y

In [9]:
from sklearn.model_selection import train_test_split

X, y = wine()
x_train,x_test,y_train, y_test = train_test_split(X , y , test_size=0.4 , random_state=0)

c1 = 0.8
c2 = 0.9
target_error = 0.00006
n_particles = 30
W = 0.5
iteration = 0 
n_iterations = 20

# get accuracy with all features
all_particle = [1 for i in range(len(x_train.columns))] # true column (feature)
all_fitness = fitness_function(np.asarray(all_particle), x_train, y_train, x_test, y_test)
print("All feature error : ", all_fitness)

particle_position = [np.random.randint(1, size=len(x_train.columns)) for _ in range(n_particles)]
pbest_position = particle_position
pbest_fitness_value = np.array([float('inf') for _ in range(n_particles)])
gbest_fitness_value = float('inf')
gbest_position = np.array([float('inf'), float('inf'), float('inf'), float('inf')])
velocity_vector = ([np.zeros(len(x_train.columns)) for _ in range(n_particles)])


while(iteration < n_iterations):
    for i in range(n_particles):
        
        fitness_cadidate = fitness_function(particle_position[i], x_train, y_train, x_test, y_test)
        # print(fitness_cadidate)
        if(pbest_fitness_value[i] > fitness_cadidate):
            pbest_fitness_value[i] = fitness_cadidate
            pbest_position[i] = particle_position[i]
        
        if(gbest_fitness_value > fitness_cadidate):
            print("update from :", gbest_fitness_value, " to ", fitness_cadidate)
            print(particle_position[i])
            gbest_fitness_value = fitness_cadidate
            gbest_position = particle_position[i]
            
        if(gbest_fitness_value == fitness_cadidate):
            print("Don't change!!!")
            
    for i in range(n_particles):
        # velocity
        new_velocity = (W*velocity_vector[i]) + (c1*random.random()) * (pbest_position[i] - particle_position[i]) + (c2*random.random()) * (gbest_position-particle_position[i])
        # position
        new_position = new_velocity + particle_position[i]
        # sigmoid
        new_position = [0 if np.random.uniform(0, 1)>= sigmoid_function(x) else 1 for x in new_position]
        particle_position[i] = np.asarray(new_position)
    
    
        
    
    print("-------> ", iteration, "<--------")
    iteration += 1

print("The best solution is : ", gbest_position, " in n_iteration: ", iteration)


20787
63674
0.9783248081841431
All feature error :  0.1271222307943787
update from : inf  to  1
[0 0 0 0 0 0 0 0 0 0 0]
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
Don't change!!!
------->  0 <--------
0.9522715659033567
update from : 1  to  0.18814417367671946
[1 1 0 1 0 0 1 1 1 0 0]
Don't change!!!
0.979862547747999
update from : 0.18814417367671946  to  0.1224980467213984
[1 0 1 1 1 1 1 0 1 0 1]
Don't change!!!
0.9767644781707681
0.9622279360016883
0.9345637262032838
0.9747254302249727
0.9605279845333242
0.9384907735464245
0.846335983602865
0.9265441496625126
0.966797611

0.7775177982907934
0.9778202755035474
------->  12 <--------
0.9395245420582494
0.9809151207821223
0.9793354260289351
0.9762851792784428
0.9788518905964056
0.9762605042016806
0.9752797553656437
0.9521293124704779
0.9838578446899824
0.9731079581723567
0.9787184920983472
0.9793997961380365
0.968905532159265
0.979883505689264
0.9044422668365413
0.9640323785008968
0.9758072869474457
0.9809151207821223
0.9778432997874478
0.9767886604106892
0.9792921236767755
0.975906691858003
0.962385136848458
0.9610206417492206
0.9777971680298451
0.9240966911346989
0.9765927319487043
0.9814018834260416
0.9461595215596161
0.9788298578889221
------->  13 <--------
0.9773397757518401
0.9767644781707681
0.9621484793415371
0.9761859422282918
0.9549367167316669
0.9291868569400015
0.9680696840139564
0.9675041245749648
0.9793138139787361
0.977727342752433
0.9464233642381454
0.9762108860755463
0.9783248081841431
0.9803686547274884
0.9705375974129933
0.975757045317831
0.976334263435275
0.9376349133660846
0.959037395

In [10]:
# # search_space = Space(1, target_error, n_particles)
# # particle_vector = [Particle() for _ in range(search_space.n_particles)]
# # search_space.particle = particle_vector
# def fitness_function(position):
#     return position[0]**2 + position[1]**2 + 1

# particle_position_vector = np.array([np.array([(-1) ** (bool(random.getrandbits(1))) * random.random()*50, (-1)**(bool(random.getrandbits(1))) * random.random()*50]) for _ in range(n_particles)])
# print(type(particle_position_vector))
# pbest_position = particle_position_vector
# pbest_fitness_value = np.array([float('inf') for _ in range(n_particles)])

# gbest_fitness_value = float('inf')
# gbest_position = np.array([float('inf'), float('inf')])

# velocity_vector = ([np.array([0, 0]) for _ in range(n_particles)])
# target = 1
# W = 0.5


# iteration = 0
# while(iteration < n_iterations):
#     for i in range(n_particles):
#         # search_space.set_pbest()
#         fitness_cadidate = fitness_function(particle_position_vector[i])
    
#         # search_space.set_gbest()
        
#         if(pbest_fitness_value[i] > fitness_cadidate):
#             pbest_fitness_value[i] = fitness_cadidate
#             pbest_position[i] = particle_position_vector[i]
        
#         if(gbest_fitness_value > fitness_cadidate):
#             gbest_fitness_value = fitness_cadidate
#             gbest_position = particle_position_vector[i]
    

#     if(abs(gbest_fitness_value - target) < target_error):
#         break
        
#     # search_space.move_particles()
#     for i in range(n_particles):
#         new_velocity = (W*velocity_vector[i]) + (c1*random.random()) * (pbest_position[i] - particle_position_vector[i]) + (c2*random.random()) * (gbest_position-particle_position_vector[i])
                
#         new_position = new_velocity + particle_position_vector[i]
#         particle_position_vector[i] = new_position
    
#     print("-------> ", iteration, "<--------")
#     iteration += 1

# print("The best solution is : ", gbest_position, " in n_iteration: ", iteration)